## Coding 2: K Nearest Neighbors



<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/220px-KnnClassification.svg.png" alt="KNN-Example" style="width: 320px;" align="right"/>The k-nearest neighbors algorithm ([KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)) is a non-parametric method.

KNN relies on the observation that (in an appropriate embedding space) similar images exist in proximity. A new test image is classified by simply looking at nearby train images and aggregating the evidence.

<img src="https://miro.medium.com/max/700/1*syyml8q8s1Yt-iEea5m1Ag.png" alt="CIFAR-10-Samples" width="50%" align="left"/>

The number of neighbors used for prediction may lead to varying outcomes. For example, the green circle (left image) may be classified either as a red triangle based on 3-nearest-neighbors or as a blue square based on the 5-nearest-neighbors.

In this exercise, we will apply KNN to classify the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset (see left image). CIFAR-10 consists of 32x32 images from 10 classes. The train set consists of 50k images and the test set consists of 10k images. The following is are sample images from each class:


### Function to load CIFAR-10.

**You do not need to fully understand this**. (Might help for later homeworks though)

`torchvision.datasets.CIFAR10` is a `Dataset` object, where `data[i]` gives you a single sample.
`torch.utils.data.DataLoader` wraps a dataset, and makes it fast (`num_workers`) as well as providing other useful features such as shuffling and batching.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

def fetch_dataloader(transform=None, batch_size=-1, is_train=True):
    """
    Loads data from disk and returns a data_loader.
    A DataLoader is similar to a list of (image, label) tuples.
    You do not need to fully understand this code to do this assignment, we're happy to explain though.
    """
    data = torchvision.datasets.CIFAR10(root='./data', train=is_train, download=True, transform=transform)
    batch = len(data) if batch_size is -1 else batch_size
    loader = torch.utils.data.DataLoader(data, batch_size=batch, shuffle=True, num_workers=4)
      
    return loader

### Fetch and preprocess data.

**You do not need to fully understand this.** What the code below does is load the training data into `x_train` of the size $N \times 3 \times 32 \times 32$ with $N$ labels `y_train` in $0 \ldots 9$.

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# -1 Batch size will give the entire dataset as one batch.
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
loader_train = fetch_dataloader(transform, batch_size=-1, is_train=True)
x_train, y_train = iter(loader_train).next()

print(x_train.shape, y_train.shape)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([50000, 3, 32, 32]) torch.Size([50000])


### Implementing KNN

Implement the `train` and `predict` functions here.

### Hints:

It is a good idea to represent the images, which are (3, 32, 32) tensors,  
as vectors of size 32768 = (3 * 32 * 32).

`torch.reshape` will be helpful in doing this.

In [ ]:
class KNearestNeighborClassifier:
    def __init__(self, k):
        self.k = k

    def train(self, x_train, y_train):
        """
        Args:
            x_train: (n, 3, 32, 32) tensor of training images
            y_train: (n) long tensor of training labels

        Implement this function to "train" your knn classifier.

        Hint: no computation is involved
        """
        self.proj = torch.nn.Linear(32*32*3, 100, bias=False).cuda()
        x1 = self.proj(x_train.view(x_train.shape[0], -1).cuda())
        x2 = self.proj(torch.flip(x_train, dims=(2,)).view(x_train.shape[0], -1).cuda())
        x3 = self.proj(torch.flip(x_train, dims=(3,)).view(x_train.shape[0], -1).cuda())
        self.x = torch.cat([x1, x2, x3], dim=0)
        self.y = torch.cat([y_train, y_train, y_train], dim=0).cuda()
        # self.x, self.y = self.proj(x_train.view(x_train.shape[0], -1).cuda()), y_train 

    def predict(self, image):
        """
        Args:
            image: (3, 32, 32) tensor
            
        Returns:
            (int) class label

        Implement this function.

        Compute distances between a test image and all train samples.
        Predict the label by voting on K nearest train samples.

        torch.topk might be useful.
        """
        # TODO: Find the distance between image and self.x
        distance = (self.x - self.proj(image.view(1, -1).cuda())).norm(dim=1)
        weights = (-5*distance).softmax(dim=0)
        one_hot_y = torch.nn.functional.one_hot(self.y, 10)
        votes = (weights[:,None] * one_hot_y).sum(dim=0)
        # print(votes)
        # print(weights.shape)
        # print(one_hot_y.shape)
        return votes.argmax()
        # _, closest = distance.topk(self.k, largest=False)
        # v, _ = self.y[closest].mode()
        # return v
        # closest = distance.argmin().cpu()
        # return self.y[closest]
        # TODO: Find the smallest distance
        # TODO: return self.y of smallest distance


K = 5

# Use a subset of the dataset for practicing implementation.
model = KNearestNeighborClassifier(k=K)
model.train(x_train, y_train)

# Uncomment when you're ready to train on the full dataset. 
# model.train(x_train, y_train)

# Test out your model on a couple samples.
for sample_idx in range(10):
    pred = model.predict(x_train[sample_idx])
    label = y_train[sample_idx]

    print(int(pred), int(label))
    # break

tensor([4.0936e-04, 1.1192e-04, 1.6873e-03, 9.9246e-01, 1.9068e-03, 8.0913e-04,
        1.3874e-03, 8.1848e-04, 2.7289e-04, 1.4079e-04], device='cuda:0',
       grad_fn=<SumBackward1>)
3 3
tensor([0.0013, 0.0015, 0.0058, 0.0041, 0.0136, 0.0033, 0.9630, 0.0045, 0.0015,
        0.0014], device='cuda:0', grad_fn=<SumBackward1>)
6 6
tensor([0.0225, 0.0064, 0.1301, 0.0351, 0.1338, 0.0383, 0.5899, 0.0242, 0.0149,
        0.0047], device='cuda:0', grad_fn=<SumBackward1>)
6 6
tensor([5.7790e-04, 5.2669e-04, 1.4718e-03, 7.0068e-04, 1.9036e-03, 6.6779e-04,
        1.4020e-03, 9.9185e-01, 4.7884e-04, 4.2698e-04], device='cuda:0',
       grad_fn=<SumBackward1>)
7 7
tensor([3.1605e-04, 2.4774e-04, 4.9872e-04, 5.5143e-04, 7.8207e-04, 7.9807e-04,
        6.4835e-04, 2.8144e-04, 9.9575e-01, 1.2908e-04], device='cuda:0',
       grad_fn=<SumBackward1>)
8 8
tensor([0.0110, 0.0042, 0.0396, 0.0109, 0.0439, 0.0143, 0.0303, 0.8345, 0.0079,
        0.0033], device='cuda:0', grad_fn=<SumBackward1>)
7 7
tensor(

### Evaluate on test data.

Play around with the hyperparameter `k` to see how it affects your accuracy.

In [ ]:
def get_predictions(model, x):
    """
    No need to change this
    """
    return torch.FloatTensor([model.predict(x_i) for x_i in x])


def compute_accuracy(y_pred, y_test):
    """
    Args:
        model: a trained KNN model
        x_pred: (10000) tensor of test predictions
        y_test: (10000) tensor of test labels

    Returns:
        (python float) representing the accuracy across the test set.

    Compute predictions on all test samples and report accuracy.
    """
    return (y_pred == y_test).float().mean()


loader_test = fetch_dataloader(transform, -1, is_train=False)
x_test, y_test = iter(loader_test).next()

print(x_test.shape, y_test.shape)

y_pred = get_predictions(model, x_test)
accuracy = compute_accuracy(y_pred, y_test)

print(accuracy)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Streaming output truncated to the last 5000 lines.
tensor([0.0462, 0.0535, 0.1041, 0.1016, 0.2448, 0.1160, 0.1252, 0.0856, 0.0830,
        0.0401], device='cuda:0', grad_fn=<SumBackward1>)
tensor([0.0372, 0.0182, 0.2169, 0.0752, 0.2403, 0.0934, 0.2338, 0.0478, 0.0264,
        0.0108], device='cuda:0', grad_fn=<SumBackward1>)
tensor([0.0868, 0.0541, 0.1227, 0.0807, 0.1596, 0.0953, 0.0991, 0.0870, 0.1486,
        0.0660], device='cuda:0', grad_fn=<SumBackward1>)
tensor([0.1891, 0.0235, 0.1032, 0.0814, 0.0923, 0.1256, 0.0524, 0.0459, 0.2668,
        0.0198], device='cuda:0', grad_fn=<SumBackward1>)
tensor([0.1274, 0.0540, 0.1068, 0.0673, 0.1771, 0.0839, 0.1036, 0.0659, 0.1731,
        0.0410], device='cuda:0', grad_fn=<SumBackward1>)
tensor([0.0414, 0.0338, 0.1773, 0.0788, 0.2672, 0.0776, 0.1830, 0.0849, 0.0280,
        0.0280], device='cuda:0', grad_fn=<SumBackward1>)
tensor([0.0982, 0.0613, 0.1357, 0.0723, 0.1763, 0.0768, 0.0904, 0.0937, 0.1273,
        0.0680], device='cuda:0', grad_fn

### References
KNN visual example is modeled after the [KNN Wikipedia article](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm).

CIFAR10 data montage is borrowed from the CIFAR10 [homepage](https://www.cs.toronto.edu/~kriz/cifar.html).

Dataloader and visualizer borrowed from the [PyTorch Tutorials](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).